In [1]:
from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply, GaussianNoise
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import Adam
from keras import losses
from keras.utils import to_categorical
import keras.backend as K

import matplotlib.pyplot as plt
import os
import numpy as np

class SGAN:
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.num_classes = 10
        self.latent_dim = 100

        optimizer = Adam(0.0002, 0.5)

        # Build and compile the discriminator
        self.discriminator = self.build_discriminator()
        self.discriminator.compile(
            loss=['binary_crossentropy', 'categorical_crossentropy'],
            loss_weights=[0.5, 0.5],
            optimizer=optimizer,
            metrics=['accuracy']
        )

        # Build the generator
        self.generator = self.build_generator()

        # The generator takes noise as input and generates imgs
        noise = Input(shape=(100,))
        img = self.generator(noise)

        # For the combined model we will only train the generator
        self.discriminator.trainable = False

        # The valid takes generated images as input and determines validity
        valid, _ = self.discriminator(img)

        # The combined model  (stacked generator and discriminator)
        # Trains generator to fool discriminator
        self.combined = Model(noise, valid)
        self.combined.compile(loss=['binary_crossentropy'], optimizer=optimizer)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=3, padding="same"))
        model.add(Activation("relu"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(1, kernel_size=3, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_discriminator(self):

        model = Sequential()

        model.add(Conv2D(32, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(128, kernel_size=3, strides=2, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Conv2D(256, kernel_size=3, strides=1, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())

        model.summary()

        img = Input(shape=self.img_shape)

        features = model(img)
        valid = Dense(1, activation="sigmoid")(features)
        label = Dense(self.num_classes+1, activation="softmax")(features)

        return Model(img, [valid, label])

    def train(self, epochs, batch_size=128, sample_interval=50):

        # Load the dataset
        (X_train, y_train), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)
        y_train = y_train.reshape(-1, 1)

        # Class weights:
        # To balance the difference in occurences of digit class labels.
        # 50% of labels that the discriminator trains on are 'fake'.
        # Weight = 1 / frequency
        half_batch = batch_size // 2
        cw1 = {0: 1, 1: 1}
        cw2 = {i: self.num_classes / half_batch for i in range(self.num_classes)}
        cw2[self.num_classes] = 1 / half_batch

        # Adversarial ground truths
        valid = np.ones((batch_size, 1))
        fake = np.zeros((batch_size, 1))

        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                global_step += 1
                imgs = X_train[index * batch_size:(index + 1) * batch_size]
                # Sample noise and generate a batch of new images
                noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                gen_imgs = self.generator.predict(noise)

                # One-hot encoding of labels
                labels = to_categorical(y_train[index * batch_size:(index + 1) * batch_size], num_classes=self.num_classes + 1)
                fake_labels = to_categorical(np.full((batch_size, 1), self.num_classes),
                                             num_classes=self.num_classes + 1)

                # Train the discriminator
                d_loss_real = self.discriminator.train_on_batch(imgs, [valid, labels], class_weight=[cw1, cw2])
                d_loss_fake = self.discriminator.train_on_batch(gen_imgs, [fake, fake_labels], class_weight=[cw1, cw2])
                d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

                # ---------------------
                #  Train Generator
                # ---------------------

                g_loss = self.combined.train_on_batch(noise, valid, class_weight=[cw1, cw2])

                # Plot the progress
                print("epoch:%d step:%d[D loss: %f, acc: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (
                epoch,global_step, d_loss[0], 100 * d_loss[3], 100 * d_loss[4], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)



    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_sgan'):
            os.mkdir('images_sgan')
        fig.savefig("images_sgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    sgan = SGAN()
    sgan.train(epochs=30, batch_size=64, sample_interval=50)



Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 14, 14, 32)        320       
_________________________________________________________________
leaky_re_lu_1 (LeakyReLU)    (None, 14, 14, 32)        0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 7, 7, 64)          18496     
_________________________________________________________________
zero_padding2d_1 (ZeroPaddin (None, 8, 8, 64)          0         
_________________________________________________________________
leaky_re_lu_2 (LeakyReLU)    (None, 8, 8, 64)          0         
_________________________________________________________________
dropout_2 (Dropout)          (None, 8, 8, 64)          0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch:0 step:1[D loss: 0.743865, acc: 25.78%, op_acc: 7.03%] [G loss: 0.931154]
epoch:0 step:2[D loss: 0.522367, acc: 70.31%, op_acc: 7.81%] [G loss: 0.918504]
epoch:0 step:3[D loss: 0.449270, acc: 82.03%, op_acc: 9.38%] [G loss: 1.277003]
epoch:0 step:4[D loss: 0.428506, acc: 81.25%, op_acc: 11.72%] [G loss: 1.479087]
epoch:0 step:5[D loss: 0.362611, acc: 94.53%, op_acc: 12.50%] [G loss: 1.378047]
epoch:0 step:6[D loss: 0.340956, acc: 92.97%, op_acc: 17.19%] [G loss: 1.664897]
epoch:0 step:7[D loss: 0.320462, acc: 93.75%, op_acc: 21.09%] [G loss: 1.834958]
epoch:0 step:8[D loss: 0.312391, acc: 96.09%, op_acc: 18.75%] [G loss: 1.690395]
epoch:0 step:9[D loss: 0.283825, acc: 95.31%, op_acc: 22.66%] [G loss: 2.099557]
epoch:0 step:10[D loss: 0.305561, acc: 93.75%, op_acc: 19.53%] [G loss: 2.402307]
epoch:0 step:11[D loss: 0.253590, acc: 98.44%, op_acc: 24.22%] [G loss: 2.180405]
epoch:0 step:12[D loss: 0.281067, acc: 95.31%, op_acc: 25.00%] [G loss: 2.117995]
epoch:0 step:13[D loss: 0.23

epoch:0 step:101[D loss: 0.363411, acc: 73.44%, op_acc: 52.34%] [G loss: 2.983675]
epoch:0 step:102[D loss: 0.236729, acc: 86.72%, op_acc: 60.16%] [G loss: 2.238795]
epoch:0 step:103[D loss: 0.185739, acc: 90.62%, op_acc: 64.84%] [G loss: 2.728579]
epoch:0 step:104[D loss: 0.258582, acc: 76.56%, op_acc: 64.84%] [G loss: 2.765528]
epoch:0 step:105[D loss: 0.218224, acc: 89.06%, op_acc: 59.38%] [G loss: 3.584298]
epoch:0 step:106[D loss: 0.349868, acc: 72.66%, op_acc: 65.62%] [G loss: 2.261792]
epoch:0 step:107[D loss: 0.242033, acc: 85.16%, op_acc: 57.03%] [G loss: 3.013606]
epoch:0 step:108[D loss: 0.613247, acc: 43.75%, op_acc: 45.31%] [G loss: 1.644354]
epoch:0 step:109[D loss: 0.429036, acc: 60.16%, op_acc: 46.88%] [G loss: 2.443802]
epoch:0 step:110[D loss: 0.376356, acc: 71.09%, op_acc: 51.56%] [G loss: 2.701674]
epoch:0 step:111[D loss: 0.348002, acc: 71.88%, op_acc: 52.34%] [G loss: 2.579358]
epoch:0 step:112[D loss: 0.348724, acc: 72.66%, op_acc: 56.25%] [G loss: 2.711916]
epoc

...

epoch:29 step:27681[D loss: 0.184833, acc: 86.72%, op_acc: 57.03%] [G loss: 1.595172]
epoch:29 step:27682[D loss: 0.306170, acc: 70.31%, op_acc: 55.47%] [G loss: 1.482905]
epoch:29 step:27683[D loss: 0.203272, acc: 90.62%, op_acc: 60.16%] [G loss: 1.236350]
epoch:29 step:27684[D loss: 0.076030, acc: 100.00%, op_acc: 72.66%] [G loss: 1.521567]
epoch:29 step:27685[D loss: 0.191728, acc: 88.28%, op_acc: 55.47%] [G loss: 1.564190]
epoch:29 step:27686[D loss: 0.086287, acc: 99.22%, op_acc: 78.91%] [G loss: 2.315699]
epoch:29 step:27687[D loss: 0.146895, acc: 96.88%, op_acc: 70.31%] [G loss: 2.140073]
epoch:29 step:27688[D loss: 0.129711, acc: 94.53%, op_acc: 74.22%] [G loss: 1.232099]
epoch:29 step:27689[D loss: 0.230084, acc: 82.81%, op_acc: 64.06%] [G loss: 1.716997]
epoch:29 step:27690[D loss: 0.233272, acc: 82.03%, op_acc: 69.53%] [G loss: 1.721600]
epoch:29 step:27691[D loss: 0.354200, acc: 60.94%, op_acc: 57.03%] [G loss: 1.645778]
epoch:29 step:27692[D loss: 0.287255, acc: 73.44%, op

epoch:29 step:27780[D loss: 0.380368, acc: 60.94%, op_acc: 63.28%] [G loss: 1.211451]
epoch:29 step:27781[D loss: 0.310864, acc: 65.62%, op_acc: 56.25%] [G loss: 1.238504]
epoch:29 step:27782[D loss: 0.384156, acc: 57.03%, op_acc: 54.69%] [G loss: 1.408655]
epoch:29 step:27783[D loss: 0.429508, acc: 52.34%, op_acc: 64.84%] [G loss: 1.586035]
epoch:29 step:27784[D loss: 0.312400, acc: 71.09%, op_acc: 59.38%] [G loss: 1.926706]
epoch:29 step:27785[D loss: 0.178738, acc: 92.19%, op_acc: 65.62%] [G loss: 1.356513]
epoch:29 step:27786[D loss: 0.283374, acc: 77.34%, op_acc: 63.28%] [G loss: 1.486351]
epoch:29 step:27787[D loss: 0.212987, acc: 91.41%, op_acc: 57.81%] [G loss: 1.298467]
epoch:29 step:27788[D loss: 0.231176, acc: 82.81%, op_acc: 57.03%] [G loss: 1.701583]
epoch:29 step:27789[D loss: 0.145907, acc: 97.66%, op_acc: 65.62%] [G loss: 2.321244]
epoch:29 step:27790[D loss: 0.540468, acc: 43.75%, op_acc: 50.78%] [G loss: 1.306904]
epoch:29 step:27791[D loss: 0.339465, acc: 63.28%, op_

epoch:29 step:27876[D loss: 0.298833, acc: 72.66%, op_acc: 64.84%] [G loss: 1.242309]
epoch:29 step:27877[D loss: 0.321524, acc: 67.19%, op_acc: 56.25%] [G loss: 0.841018]
epoch:29 step:27878[D loss: 0.423361, acc: 48.44%, op_acc: 54.69%] [G loss: 2.021201]
epoch:29 step:27879[D loss: 0.138739, acc: 94.53%, op_acc: 75.78%] [G loss: 2.397720]
epoch:29 step:27880[D loss: 0.118714, acc: 96.09%, op_acc: 70.31%] [G loss: 1.479138]
epoch:29 step:27881[D loss: 0.098318, acc: 99.22%, op_acc: 73.44%] [G loss: 2.709405]
epoch:29 step:27882[D loss: 0.116272, acc: 93.75%, op_acc: 68.75%] [G loss: 2.459285]
epoch:29 step:27883[D loss: 0.288891, acc: 69.53%, op_acc: 68.75%] [G loss: 1.564502]
epoch:29 step:27884[D loss: 0.270447, acc: 75.78%, op_acc: 68.75%] [G loss: 1.582353]
epoch:29 step:27885[D loss: 0.258051, acc: 78.12%, op_acc: 72.66%] [G loss: 1.216076]
epoch:29 step:27886[D loss: 0.202382, acc: 80.47%, op_acc: 53.91%] [G loss: 1.444876]
epoch:29 step:27887[D loss: 0.185081, acc: 85.94%, op_

epoch:29 step:27976[D loss: 0.202581, acc: 85.16%, op_acc: 73.44%] [G loss: 1.936159]
epoch:29 step:27977[D loss: 0.266322, acc: 76.56%, op_acc: 57.81%] [G loss: 1.709716]
epoch:29 step:27978[D loss: 0.206592, acc: 92.19%, op_acc: 64.06%] [G loss: 1.792580]
epoch:29 step:27979[D loss: 0.175454, acc: 89.06%, op_acc: 57.81%] [G loss: 1.662978]
epoch:29 step:27980[D loss: 0.521567, acc: 35.94%, op_acc: 51.56%] [G loss: 0.672488]
epoch:29 step:27981[D loss: 0.191351, acc: 87.50%, op_acc: 63.28%] [G loss: 1.869096]
epoch:29 step:27982[D loss: 0.136078, acc: 97.66%, op_acc: 66.41%] [G loss: 1.949158]
epoch:29 step:27983[D loss: 0.159379, acc: 92.19%, op_acc: 66.41%] [G loss: 2.681544]
epoch:29 step:27984[D loss: 0.446275, acc: 55.47%, op_acc: 57.81%] [G loss: 1.559859]
epoch:29 step:27985[D loss: 0.383049, acc: 53.12%, op_acc: 54.69%] [G loss: 1.326138]
epoch:29 step:27986[D loss: 0.243872, acc: 83.59%, op_acc: 59.38%] [G loss: 1.170922]
epoch:29 step:27987[D loss: 0.163728, acc: 92.97%, op_

epoch:29 step:28075[D loss: 0.526606, acc: 35.94%, op_acc: 52.34%] [G loss: 1.436636]
epoch:29 step:28076[D loss: 0.254658, acc: 82.03%, op_acc: 63.28%] [G loss: 1.508063]
epoch:29 step:28077[D loss: 0.090005, acc: 100.00%, op_acc: 74.22%] [G loss: 2.571362]
epoch:29 step:28078[D loss: 0.341719, acc: 62.50%, op_acc: 59.38%] [G loss: 1.628353]
epoch:29 step:28079[D loss: 0.076848, acc: 98.44%, op_acc: 75.78%] [G loss: 2.370924]
epoch:29 step:28080[D loss: 0.315243, acc: 63.28%, op_acc: 73.44%] [G loss: 1.800461]
epoch:29 step:28081[D loss: 0.460201, acc: 45.31%, op_acc: 55.47%] [G loss: 1.428455]
epoch:29 step:28082[D loss: 0.333117, acc: 67.19%, op_acc: 57.03%] [G loss: 1.502421]
epoch:29 step:28083[D loss: 0.095528, acc: 93.75%, op_acc: 60.94%] [G loss: 1.589963]
epoch:29 step:28084[D loss: 0.278731, acc: 71.88%, op_acc: 60.16%] [G loss: 1.262039]
epoch:29 step:28085[D loss: 0.099922, acc: 96.88%, op_acc: 64.84%] [G loss: 2.420795]
epoch:29 step:28086[D loss: 0.432188, acc: 50.78%, op